In [1]:
import pandas as pd
import numpy as np
from haversine import haversine

,num, time, latitude, longitude, sog, cog, hdg, anchor_drag, time_gap, d_km/s, dd_km/s

In [2]:
busan_test = pd.read_csv("../기상청공모전/data/busan_anch_drag_test.csv")
busan_test = busan_test[['busan_anch_drag_test.time','busan_anch_drag_test.num', 'busan_anch_drag_test.lat','busan_anch_drag_test.lon', 'busan_anch_drag_test.sog','busan_anch_drag_test.cog', 'busan_anch_drag_test.hdg']]
busan_test.columns = ['time', 'num' ,'latitude','longitude','sog','cog','hdg']
busan_test = busan_test[['num','time','latitude','longitude','sog','cog','hdg']]
busan_test = busan_test.replace(regex=r'["]', value='')
busan_test = busan_test.replace(regex=r'[N]', value='')
busan_test = busan_test.replace(regex=r'[E]', value='')
busan_test['time'] = pd.to_datetime(busan_test['time'])
busan_test['latitude'] = busan_test['latitude'].astype(float)
busan_test['longitude'] = busan_test['longitude'].astype(float)
busan_test['time_gap'] = 0
busan_test['d_km/s'] = 0
busan_test['dd_km/s'] = 0
busan_test


,num,time,latitude,longitude,sog,cog,hdg,time_gap,d_km/s,dd_km/s
0,1001,2022-08-14 00:55:06,35.052557,129.040953,0.7,67.4,208,0,0,0
1,1001,2022-08-14 00:55:16,35.052587,129.041017,1.1,61.3,208,0,0,0
2,1001,2022-08-14 00:55:24,35.052625,129.041087,1.5,57.7,207,0,0,0
3,1001,2022-08-14 00:55:36,35.052680,129.041187,1.8,56.2,208,0,0,0
4,1001,2022-08-14 00:55:45,35.052732,129.041275,2.0,55.6,209,0,0,0
...,...,...,...,...,...,...,...,...,...,...
153294,1116,2023-03-08 11:01:43,35.021717,129.065633,0.4,157.0,220,0,0,0
153295,1116,2023-03-08 11:02:04,35.021667,129.065650,0.4,153.0,219,0,0,0
153296,1116,2023-03-08 11:02:12,35.021650,129.065667,0.4,150.0,219,0,0,0
153297,1116,2023-03-08 11:02:22,35.021617,129.065683,0.4,148.0,218,0,0,0


In [3]:
for i in range(1, len(busan_test)):
    if busan_test.loc[i, 'time_gap'] == 0:
        dt = (pd.to_datetime(busan_test.loc[i, 'time']) - pd.to_datetime(busan_test.loc[i-1, 'time'])).total_seconds()
        if dt != 0:
            busan_test.loc[i, 'time_gap'] = dt
        else:
            a = busan_test.loc[(busan_test['num'] == busan_test.loc[i, 'num']) & (busan_test['time'] == busan_test.loc[i, 'time'])].index.tolist()
            busan_test.loc[a, 'time_gap'] = 1 / len(a)

# 결과 출력
print(busan_test)


         num                time   latitude   longitude  sog    cog  hdg  \
0       1001 2022-08-14 00:55:06  35.052557  129.040953  0.7   67.4  208   
1       1001 2022-08-14 00:55:16  35.052587  129.041017  1.1   61.3  208   
2       1001 2022-08-14 00:55:24  35.052625  129.041087  1.5   57.7  207   
3       1001 2022-08-14 00:55:36  35.052680  129.041187  1.8   56.2  208   
4       1001 2022-08-14 00:55:45  35.052732  129.041275  2.0   55.6  209   
...      ...                 ...        ...         ...  ...    ...  ...   
153294  1116 2023-03-08 11:01:43  35.021717  129.065633  0.4  157.0  220   
153295  1116 2023-03-08 11:02:04  35.021667  129.065650  0.4  153.0  219   
153296  1116 2023-03-08 11:02:12  35.021650  129.065667  0.4  150.0  219   
153297  1116 2023-03-08 11:02:22  35.021617  129.065683  0.4  148.0  218   
153298  1116 2023-03-08 11:02:54  35.021550  129.065733  0.5  142.0  217   

        time_gap  d_km/s  dd_km/s  
0            0.0       0        0  
1           10.

In [4]:
for i in range(1, len(busan_test)):
    busan_test.loc[i, 'd_km/s'] = haversine((busan_test.loc[i, 'latitude'],busan_test.loc[i, 'longitude']),(busan_test.loc[i-1, 'latitude'],busan_test.loc[i-1, 'longitude']),unit = 'km') / busan_test.loc[i, 'time_gap']

In [5]:
for i in range(2, len(busan_test)):
    busan_test.loc[i, 'dd_km/s'] = (busan_test.loc[i, 'd_km/s'] - busan_test.loc[i-1, 'd_km/s']) / busan_test.loc[i,'time_gap']

In [8]:
# 각 배 번호의 1번째 2번째 삭제
ship_num_busan = busan_test['num'].unique()

_drop_busan = []
for i in ship_num_busan:
    k = busan_test[busan_test["num"] == i].index[0]
    _drop_busan.append(k)
    k += 1
    _drop_busan.append(k)
    
busan_test = busan_test[~busan_test.index.isin(_drop_busan)].reset_index(drop=True)

In [9]:
busan_test

,num,time,latitude,longitude,sog,cog,hdg,time_gap,d_km/s,dd_km/s
0,1001,2022-08-14 00:55:24,35.052625,129.041087,1.5,57.7,207,8.0,0.000956,3.554742e-05
1,1001,2022-08-14 00:55:36,35.052680,129.041187,1.8,56.2,208,12.0,0.000914,-3.486275e-06
2,1001,2022-08-14 00:55:45,35.052732,129.041275,2.0,55.6,209,9.0,0.001098,2.042536e-05
3,1001,2022-08-14 00:56:06,35.052848,129.041488,2.1,56.0,212,21.0,0.001109,5.345358e-07
4,1001,2022-08-14 00:56:16,35.052908,129.041600,2.1,56.5,214,10.0,0.001218,1.094812e-05
...,...,...,...,...,...,...,...,...,...,...
153062,1116,2023-03-08 11:01:43,35.021717,129.065633,0.4,157.0,220,20.0,0.000197,-1.235655e-06
153063,1116,2023-03-08 11:02:04,35.021667,129.065650,0.4,153.0,219,21.0,0.000275,3.686469e-06
153064,1116,2023-03-08 11:02:12,35.021650,129.065667,0.4,150.0,219,8.0,0.000305,3.823985e-06
153065,1116,2023-03-08 11:02:22,35.021617,129.065683,0.4,148.0,218,10.0,0.000395,8.939784e-06


In [10]:
busan_test.to_csv('../기상청공모전/data/busan_test(der).csv')